In [1]:
import numpy as np
import math
import pickle
import os

In [2]:
def DPAlgo(M,N,T,alpha,pi,R,ind,loud):

  # M is the initial value of M
  # N is the number of shares we have to sell
  # T is the number of timesteps
  # alpha, pi are parameters
  # R is the DP array: R[t][m][n] is the maximum amount of shares we can sell if we are at time t, the previous M was m and we have n shares to sell
  # ind[t][m][n] is the number of shares we should try to sell if we are at time t, the previous M was m and we have n shares to sell

  iarray = np.array(range(N+1)).astype(int)
  impact = 1 - alpha*iarray**pi # impact[i] = 1-alpha*i^pi
  marray = iarray.reshape(-1, 1)

  # For some values of alpha, the impact may become negative.
  # This causes S_t to be negative, so we don't sell anything in those cases.
  impact[impact < 0] = 0

  # Boundary conditions at T-1: try to sell everything
  for x in range(N+1):
    for y in range(N+1):
      R[T-1][x][y] = math.ceil(y * impact[math.ceil(0.1 * x + 0.9 * y)])
      ind[T-1][x][y] = y

  candidates = np.zeros((N+1,N+1)) #used to keep the 'candidates'

  # We will loop through every timestep going from T-1 backwards.
  # For each possible n (number of shares left to sell), we will have:
    # A 2D array StVec: StVec[m][x] is the value of S_t if the previous value of M is m and we try to sell x<=n shares
    # A 2D array futureRewardVec: futureRewardVec[m][x] is the maximum amount of shares we can sell from the next timestep onward,
    #   if the current previous M is m and we try to sell x<=n now
  # The 2D array candidates holds the sum of the two arrays above.
  # Ultimately, we need for each value of m, the optimal amount x.
  for k in range(1, T):
      t = T - k - 1
      # if loud: print("t = ",t)
      for n in range(N+1):
        StVec = np.ceil(iarray[:n+1] * impact[np.ceil(0.1 * marray + 0.9 * iarray[:n+1]).astype(int)])
        futureRewardVec = R[t+1, np.ceil(0.1 * marray + 0.9 * iarray[:n+1]).astype(int), n - iarray[:n+1]]
        np.copyto(candidates[:,:n+1], StVec + futureRewardVec)
        ind[t, :, n]=np.argmax(candidates[:, :n+1], axis=1)
        R[t, iarray, n] = candidates[iarray, ind[t, iarray, n]]
  if loud: print('optimal Revenue:', R[0, M, N])

def getSched(M,N,T,ind):

  # Function to extract the schedule from the 3D array ind

  sched = np.zeros(T) # We will keep the schedule here
  prevM = M
  currN = N
  for t in range(T):
    print("At time", t)
    print("We try to sell ", ind[t][prevM][currN], " shares")

    sched[t]=ind[t][prevM][currN]
    currM = math.ceil(0.1*prevM+0.9*ind[t,prevM,currN])

    print("Current M is ", currM)
    print("We mange to sell ", math.ceil(ind[t][prevM][currN]*(1 - alpha*currM**pi)))
    print("The algorithm predicts ",R[t,prevM,currN])

    currN = (currN-ind[t][prevM][currN]).astype(int)
    prevM = currM

    print("We have ", currN, " left")

  return sched

# Approximate solution for large N -> sell only multiples of 100
# The value of M we use is M/100.

def DPAlgoApprox(M,N,T,skip,alpha,pi,R,ind,loud):

  iarray = np.array(range(N//skip+1)).astype(int)
  impact = 1 - alpha*(skip * iarray)**pi # impact[i] = 1-alpha*i^pi
  marray = iarray.reshape(-1, 1)

  impact[impact < 0] = 0

  for x in range(N//skip+1):
    for y in range(N//skip+1):
      R[T-1][x][y] = math.ceil(skip * y * impact[math.ceil(0.1 * skip * x + 0.9 * skip * y)//skip])
      ind[T-1][x][y] = y

  candidates = np.zeros((N//skip+1,N//skip+1))

  for k in range(1, T):
      t = T - k - 1
      if loud: print("t = ",t)
      for n in range(N//skip+1):
        StVec = np.ceil(skip * iarray[:n+1] * impact[np.ceil(0.1 * 100 * marray + 0.9 * 100 * iarray[:n+1]).astype(int)//skip])
        futureRewardVec = R[t+1, np.ceil(0.1 * 100 * marray + 0.9 * 100 * iarray[:n+1]).astype(int)//skip, n - iarray[:n+1]]
        np.copyto(candidates[:,:n+1], StVec + futureRewardVec)
        ind[t, :, n]=np.argmax(candidates[:, :n+1], axis=1)
        R[t, iarray, n] = candidates[iarray, ind[t, iarray, n]]
  if loud: print('optimal Revenue:', R[0, M//skip, N//skip])

def getSchedApprox(M,N,T,skip,pi,alpha,ind):

  sched = np.zeros(T)
  prevM = M//skip
  currN = N//skip
  for t in range(T):
    sched[t]=skip * ind[t][prevM][currN]
    currM = math.ceil(0.1 * skip * prevM + 0.9 * skip * ind[t,prevM,currN])//skip
    currN = (currN-ind[t][prevM][currN]).astype(int)
    prevM = currM

  return sched


In [3]:
# # Example run

# M = 0
# N = int(1e3)
# T = 10
# pi = .5
# alpha = 1e-3
# R = np.zeros((T,N+1,N+1))
# ind = np.zeros((T,N+1,N+1)).astype(int)
# DPAlgo(M, N, T, alpha, pi, R, ind, True)

# # Runs for 4.6s with (500, 4)
# # Runs for 9.5s with (500, 10)
# # Runs for 22.8s with (1000, 4) -> 387 with schedule array([238., 256., 242., 264.])
# # Runs for 63.2s with (1000, 10) -> 695 with schedule array([ 95.,  97.,  97.,  97.,  97.,  97.,  97.,  97., 108., 118.])

# sched = getSched(M, N, T, ind)

M = 0
N = int(1e5)
T = 10
pi = .5
alpha = 1e-3
skip = 100
R = np.zeros((T,N//skip+1,N//skip+1)) # -> R[time][M][shares]
ind = np.zeros((T,N//skip+1,N//skip+1)).astype(int)
DPAlgoApprox(M,N,T,skip,alpha,pi,R,ind, True)

sched = getSchedApprox(M,N,T,skip,pi,alpha,ind)

t =  8
t =  7
t =  6
t =  5
t =  4
t =  3
t =  2
t =  1
t =  0
optimal Revenue: 90113.0


In [11]:
data_dir = './data'
if not os.path.exists(data_dir): os.makedirs(data_dir)
with open(f'{ data_dir }/orlin.pkl', 'wb') as f:
    pickle.dump(sched, f)